### This notebook convertes  "Who left?" Figure (figure-2) R script code to python. 

Author: Mariela Rosa Casillas & Camila Vazquez Rodriguez 

## We Installed 

In [ ]:
! pip3 install pandas 
! pip3 install pyreadr 
! pip3 install numpy 
! pip3 install matplotlib
! pip3 install plotnine 


### Pyreadr to load R data files with dataframes 
### Pandas to handle dataframes 
### Numpy for math operations
### Matplotlib to plot 
### And plotnine, because it hass ggplot, to plot 

# Imports 

### R

### Python

In [ ]:
import pandas as pd
import pyreadr
import numpy as np
import plotnine as p9
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap, geom_bar

# Load Data 

### R

### Python 

In [ ]:
# Load R files 
load_data = pyreadr.read_r('../data/rdata/ACS2016.Rdata')
individuals = pyreadr.read_r("../data/rdata/ind_hh.RDS")
where_df = pyreadr.read_r("../data/rdata/where_df.RDS")
hh_main = pyreadr.read_r("../data/rdata/hh_main.RDS")

# Convert to Pandas Dataframe
individuals_df=individuals[None]
where_df = where_df[None]
hh_main = hh_main[None]


#### Lets see the data

In [ ]:
individuals_df.head()

In [ ]:
where_df.head()

In [ ]:
hh_main.head()

 # Working with the data

### R 

### Python

 #### Merge individuals with HH strata

In [ ]:
ind_hh = individuals_df.merge(hh_main, on=['hh_id', 'strata'], how='left')

In [ ]:
factors = {'0': "Don't Know",
 '1': 'Somewhere else in PR',
 '2': 'Florida',
 '3': 'New York',
 '4': 'Texas',
 '5': 'Another State',
 '6': 'Another Country',
 '7': "Don't Know",
 '---': None}
ind_hh = ind_hh.assign(left_final = ind_hh['left_final'].map(lambda x: factors[x]))

In [ ]:
#ind_hh['left_final'].unique()

In [ ]:
temp = ind_hh.assign(left = ind_hh['left_final'].map(lambda x: 0 if x != None else 1))

In [ ]:
temp['age_cat'] = pd.cut(temp.age, list(range(0,85,10)) + [np.infty], include_lowest = True, right = False)

#### We coulnt find the equivelent of all the transformations done in the data so we decided to group by age_cat and left_final car 

In [ ]:
left_df = temp.groupby(["age_cat", "left_final_cat"])

In [ ]:
left_df = left_df.size().unstack()

In [ ]:
left_df.columns

# Plotting

### R 

### Python

#### We did a simple plot with matplotlib

In [ ]:
left_df.plot(kind='bar',stacked=True)

#### Now using ggplot

In [ ]:
from plotnine import *
import pandas as pd


In [ ]:
left_df.columns=left_df.columns.astype('str')
left_df.reset_index()

In [ ]:
left_df = temp.groupby(["age_cat", "left_final_cat"])

In [ ]:
ggplot(temp) + geom_bar(aes(x='age_cat'))


In [ ]:
ggplot(temp, aes(x="age_cat", fill="left_final_cat")) + geom_bar()

In [ ]:
ggplot(temp, aes(x="age_cat", fill="left_final_cat")) + geom_bar( alpha = .85, width = .45)

In [ ]:
ggplot(temp, aes(x="age_cat", fill="left_final_cat")) + geom_bar(position='fill', alpha = .85, width = .45)